In [0]:
import pandas as pd

In [0]:

# read using panda and convert into spark dataframe
for year in range(2010,2016):
  url = f"https://raw.githubusercontent.com/databricks/Spark-The-Definitive-Guide/master/data/flight-data/csv/{year}-summary.csv"
  if year > 2010:
    df = pd.concat([df, pd.read_csv(url)], axis=0)
  else:
    df = pd.read_csv(url)

flight_df = spark.createDataFrame(df)

In [0]:
flight_df.count(), df.shape

Out[25]: (1502, (1502, 3))

In [0]:
# reading using spark itself
from pyspark import SparkFiles
def read_spark_csv(url,year):
  csvname = f"{year}-summary.csv"
  filename = "file://"+SparkFiles.get(csvname)
  return spark.read.csv(filename,header=True,inferSchema=True)


for year in range(2010,2016):
  url = f"https://raw.githubusercontent.com/databricks/Spark-The-Definitive-Guide/master/data/flight-data/csv/{year}-summary.csv"
  spark.sparkContext.addFile(url)
  if year > 2010:
    flight_df = flight_df.union(read_spark_csv(url,year))
  else:
    flight_df = read_spark_csv(url,year)



In [0]:
flight_df.count(), len(flight_df.columns)

Out[33]: (1502, 3)

In [0]:
# we can also read csv files like :
# flightData2015 = spark\
# .read\
# .option("inferSchema", "true")\
# .option("header", "true")\
# .csv("/data/flight-data/csv/2015-summary.csv")

In [0]:
flight_df.take(3)

Out[35]: [Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=264),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='India', count=69)]

In [0]:
flight_df.head(3)

Out[37]: [Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=264),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='India', count=69)]

In [0]:
flight_df.printSchema()

root
-- DEST_COUNTRY_NAME: string (nullable = true)
-- ORIGIN_COUNTRY_NAME: string (nullable = true)
-- count: integer (nullable = true)

In [0]:
flight_df.sort('count').explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [count#249 ASC NULLS FIRST], true, 0
 +- Exchange rangepartitioning(count#249 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [id=#996]
 +- Union
 :- FileScan csv [DEST_COUNTRY_NAME#247,ORIGIN_COUNTRY_NAME#248,count#249] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/local_disk0/spark-3e429e81-4605-4229-9cf8-964a17f963ab/userFiles-f4720aa3..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>
 :- FileScan csv [DEST_COUNTRY_NAME#269,ORIGIN_COUNTRY_NAME#270,count#271] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/local_disk0/spark-3e429e81-4605-4229-9cf8-964a17f963ab/userFiles-f4720aa3..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>
 :- FileScan csv [DEST_COUNTRY_NAME#294,ORIGIN_COUNTRY_NAME#295,count#296] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/local_disk0/spark-3e429e81-4605-4229-9cf8-964a17f963ab/userFiles-f4720aa3..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>
 :- FileScan csv [DEST_COUNTRY_NAME#319,ORIGIN_COUNTRY_NAME#320,count#321] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/local_disk0/spark-3e429e81-4605-4229-9cf8-964a17f963ab/userFiles-f4720aa3..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>
 :- FileScan csv [DEST_COUNTRY_NAME#344,ORIGIN_COUNTRY_NAME#345,count#346] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/local_disk0/spark-3e429e81-4605-4229-9cf8-964a17f963ab/userFiles-f4720aa3..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>
 +- FileScan csv [DEST_COUNTRY_NAME#369,ORIGIN_COUNTRY_NAME#370,count#371] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/local_disk0/spark-3e429e81-4605-4229-9cf8-964a17f963ab/userFiles-f4720aa3..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>

In [0]:
spark.conf.set('spark.sql.shuffle.partitions','5')

In [0]:
output_df = flight_df.sort('count')

In [0]:
output_df.rdd.getNumPartitions()

Out[44]: 1

In [0]:
flight_df.sort('count').explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [count#249 ASC NULLS FIRST], true, 0
 +- Exchange rangepartitioning(count#249 ASC NULLS FIRST, 5), ENSURE_REQUIREMENTS, [id=#1133]
 +- Union
 :- FileScan csv [DEST_COUNTRY_NAME#247,ORIGIN_COUNTRY_NAME#248,count#249] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/local_disk0/spark-3e429e81-4605-4229-9cf8-964a17f963ab/userFiles-f4720aa3..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>
 :- FileScan csv [DEST_COUNTRY_NAME#269,ORIGIN_COUNTRY_NAME#270,count#271] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/local_disk0/spark-3e429e81-4605-4229-9cf8-964a17f963ab/userFiles-f4720aa3..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>
 :- FileScan csv [DEST_COUNTRY_NAME#294,ORIGIN_COUNTRY_NAME#295,count#296] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/local_disk0/spark-3e429e81-4605-4229-9cf8-964a17f963ab/userFiles-f4720aa3..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>
 :- FileScan csv [DEST_COUNTRY_NAME#319,ORIGIN_COUNTRY_NAME#320,count#321] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/local_disk0/spark-3e429e81-4605-4229-9cf8-964a17f963ab/userFiles-f4720aa3..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>
 :- FileScan csv [DEST_COUNTRY_NAME#344,ORIGIN_COUNTRY_NAME#345,count#346] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/local_disk0/spark-3e429e81-4605-4229-9cf8-964a17f963ab/userFiles-f4720aa3..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>
 +- FileScan csv [DEST_COUNTRY_NAME#369,ORIGIN_COUNTRY_NAME#370,count#371] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/local_disk0/spark-3e429e81-4605-4229-9cf8-964a17f963ab/userFiles-f4720aa3..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>

In [0]:
flight_df.createOrReplaceTempView('flight_table')

In [0]:
destination_in_edges = spark.sql("""

      select DEST_COUNTRY_NAME, count(1)
      from flight_table
      group by DEST_COUNTRY_NAME;



""")

In [0]:
destination_in_edges.show(3)

+--------------------+--------+
 DEST_COUNTRY_NAME|count(1)|
+--------------------+--------+
 Equatorial Guinea| 2|
 Bolivia| 6|
Turks and Caicos ...| 6|
+--------------------+--------+
only showing top 3 rows

In [0]:
destination_df = flight_df.groupBy("DEST_COUNTRY_NAME").count()

In [0]:
destination_df.show(3)

+--------------------+-----+
 DEST_COUNTRY_NAME|count|
+--------------------+-----+
 Equatorial Guinea| 2|
 Bolivia| 6|
Turks and Caicos ...| 6|
+--------------------+-----+
only showing top 3 rows

In [0]:
flight_df.groupBy("DEST_COUNTRY_NAME").count().explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[DEST_COUNTRY_NAME#247], functions=[finalmerge_count(merge count#688L) AS count(1)#683L])
 +- Exchange hashpartitioning(DEST_COUNTRY_NAME#247, 5), ENSURE_REQUIREMENTS, [id=#1533]
 +- HashAggregate(keys=[DEST_COUNTRY_NAME#247], functions=[partial_count(1) AS count#688L])
 +- Union
 :- FileScan csv [DEST_COUNTRY_NAME#247] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/local_disk0/spark-3e429e81-4605-4229-9cf8-964a17f963ab/userFiles-f4720aa3..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>
 :- FileScan csv [DEST_COUNTRY_NAME#269] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/local_disk0/spark-3e429e81-4605-4229-9cf8-964a17f963ab/userFiles-f4720aa3..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>
 :- FileScan csv [DEST_COUNTRY_NAME#294] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/local_disk0/spark-3e429e81-4605-4229-9cf8-964a17f963ab/userFiles-f4720aa3..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>
 :- FileScan csv [DEST_COUNTRY_NAME#319] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/local_disk0/spark-3e429e81-4605-4229-9cf8-964a17f963ab/userFiles-f4720aa3..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>
 :- FileScan csv [DEST_COUNTRY_NAME#344] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/local_disk0/spark-3e429e81-4605-4229-9cf8-964a17f963ab/userFiles-f4720aa3..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>
 +- FileScan csv [DEST_COUNTRY_NAME#369] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/local_disk0/spark-3e429e81-4605-4229-9cf8-964a17f963ab/userFiles-f4720aa3..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>

In [0]:
spark.sql("""

      select DEST_COUNTRY_NAME, count(1)
      from flight_table
      group by DEST_COUNTRY_NAME;



""").explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[DEST_COUNTRY_NAME#247], functions=[finalmerge_count(merge count#697L) AS count(1)#692L])
 +- Exchange hashpartitioning(DEST_COUNTRY_NAME#247, 5), ENSURE_REQUIREMENTS, [id=#1615]
 +- HashAggregate(keys=[DEST_COUNTRY_NAME#247], functions=[partial_count(1) AS count#697L])
 +- Union
 :- FileScan csv [DEST_COUNTRY_NAME#247] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/local_disk0/spark-3e429e81-4605-4229-9cf8-964a17f963ab/userFiles-f4720aa3..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>
 :- FileScan csv [DEST_COUNTRY_NAME#269] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/local_disk0/spark-3e429e81-4605-4229-9cf8-964a17f963ab/userFiles-f4720aa3..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>
 :- FileScan csv [DEST_COUNTRY_NAME#294] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/local_disk0/spark-3e429e81-4605-4229-9cf8-964a17f963ab/userFiles-f4720aa3..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>
 :- FileScan csv [DEST_COUNTRY_NAME#319] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/local_disk0/spark-3e429e81-4605-4229-9cf8-964a17f963ab/userFiles-f4720aa3..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>
 :- FileScan csv [DEST_COUNTRY_NAME#344] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/local_disk0/spark-3e429e81-4605-4229-9cf8-964a17f963ab/userFiles-f4720aa3..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>
 +- FileScan csv [DEST_COUNTRY_NAME#369] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/local_disk0/spark-3e429e81-4605-4229-9cf8-964a17f963ab/userFiles-f4720aa3..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>

In [0]:
#  both of them has same execution plan
flight_df.printSchema()

root
-- DEST_COUNTRY_NAME: string (nullable = true)
-- ORIGIN_COUNTRY_NAME: string (nullable = true)
-- count: integer (nullable = true)

In [0]:
# using dataframe
from pyspark.sql.functions import max
import pyspark.sql.functions as F

flight_df.orderBy(F.desc('count')).take(1)

Out[64]: [Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='United States', count=370002)]

In [0]:
# using sql
spark.sql("""

    select DEST_COUNTRY_NAME, ORIGIN_COUNTRY_NAME, count
    from flight_table
    order by count DESC
    limit 1;


""").take(1)

Out[70]: [Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='United States', count=370002)]

In [0]:
from pyspark.sql.functions import desc

#  using dataframe
flight_df\
  .groupBy('DEST_COUNTRY_NAME')\
  .sum('count')\
  .withColumnRenamed('sum(count)','destination_total')\
  .sort(desc('destination_total'))\
  .limit(5)\
  .show()
        

+-----------------+-----------------+
DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
 United States| 2348280|
 Canada| 49052|
 Mexico| 38075|
 United Kingdom| 10946|
 Japan| 9205|
+-----------------+-----------------+

In [0]:
# using sql

spark.sql("""
    
    select DEST_COUNTRY_NAME, sum(count) as DESTINATION_COUNT
    from flight_table
    group by DEST_COUNTRY_NAME
    order by DESTINATION_COUNT DESC
    limit 5;



""").show()

+-----------------+-----------------+
DEST_COUNTRY_NAME|DESTINATION_COUNT|
+-----------------+-----------------+
 United States| 2348280|
 Canada| 49052|
 Mexico| 38075|
 United Kingdom| 10946|
 Japan| 9205|
+-----------------+-----------------+